## Transform Step

In [19]:
role = "<Execution_Role>"

In [7]:
!mkdir -p data

In [ ]:
import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name

In [ ]:
local_path = "data/abalone-dataset-batch"

s3 = boto3.resource("s3")
s3.Bucket(f"sagemaker-servicecatalog-seedcode-{region}").download_file(
    "dataset/abalone-dataset-batch", local_path
)

base_uri = f"s3://<Bucket_Name>/abalone"  ## replace bucket_name
batch_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=base_uri,
)
print(batch_data_uri)

In [10]:
from sagemaker.workflow.parameters import (
    ParameterString,
    ParameterFloat,
)

batch_data = ParameterString(
    name="BatchData",
    default_value=batch_data_uri,
)

mse_threshold = ParameterFloat(name="MseThreshold", default_value=6.0)

In [12]:
from sagemaker.transformer import Transformer


transformer = Transformer(
    model_name="trained-model2023-06-26-02-10-38",
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path=f"s3://<Bucket_Name>/AbaloneTransform", ## replace bucket_name
)

In [13]:
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep


step_transform = TransformStep(
    name="BTStep", transformer=transformer, inputs=TransformInput(data=batch_data)
)

## Define a Pipeline of Parameters, Steps, and Conditions

In this section, combine the steps into a Pipeline so it can be executed.

A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair.

Note:

* All the parameters used in the definitions must be present.
* Steps passed into the pipeline do not have to be listed in the order of execution. The SageMaker Pipeline service resolves the data dependency DAG as steps for the execution to complete.
* Steps must be unique to across the pipeline step list and all condition step if/else lists.

In [15]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = f"BTPipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        batch_data,
        mse_threshold,
    ],
    steps=[step_transform]
)

In [ ]:
import json

definition = json.loads(pipeline.definition())
definition

## Submit the pipeline to SageMaker and start execution

Submit the pipeline definition to the Pipeline service. The Pipeline service uses the role that is passed in to create all the jobs defined in the steps.

##### Note that this only needs to run 1 time as the update of the pipline will be done in Lambda function. (unless some parameters has been changed)

In [ ]:
pipeline.upsert(role_arn=role)

Start the pipeline and accept all the default parameters. 
Once the Lambda function is triggered, the only thing that needs to be executed is this line of code.

In [22]:
execution = pipeline.start()